In [11]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
# Create fully connected network

class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1= nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, input_size)

  def forward(self, x):
    x = f.relu(self.fc1((x)))
    x = self.fc2(x)
    return x

In [7]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {}".format(device))

using cuda


In [8]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [9]:
#Load data
train_dataset = datasets.MNIST(root='database/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='database/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting database/MNIST/raw/train-images-idx3-ubyte.gz to database/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting database/MNIST/raw/train-labels-idx1-ubyte.gz to database/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting database/MNIST/raw/t10k-images-idx3-ubyte.gz to database/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting database/MNIST/raw/t10k-labels-idx1-ubyte.gz to database/MNIST/raw



In [10]:
#Initialize network
model = NN(input_size=input_size, num_classes= num_classes).to(device)

In [13]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [14]:
# Train network
for epoch in range(num_epochs):
  for batch, (data, targets) in enumerate(train_loader):
    # get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # get data to correct shape
    data = data.reshape(data.shape[0], -1)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # optimizer step
    optimizer.step()

In [19]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checkin on the train data")
  else:
    print("Checkin on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, pred = scores.max(1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0)
  
    print(f'got {num_correct} / {num_samples} with {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()

In [20]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checkin on the train data
got 55887 / 60000 with 93.14
Checkin on test data
got 9266 / 10000 with 92.66
